In [1]:
!pip install pandas
!pip install sklearn

In [2]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import numpy as np

df_train=pd.read_csv("/content/drive/MyDrive/train.csv")
df_test=pd.read_csv("/content/drive/MyDrive/test.csv")

In [3]:
df_train.head(66)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN
62,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,36973,83.4750,C83,S
63,64,0,3,"Skoog, Master. Harald",male,4.0,3,2,347088,27.9000,NaN,S
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C


In [4]:
# Function to get the mean age...

def mean_age(list_of_ages):
  sum_of_ages = sum(list_of_ages)
  mean_of_ages = sum_of_ages / len(list_of_ages)
  return round(mean_of_ages, 0)

In [5]:
# Function to replace nan age values with the mean age of people who share the same title (e.g. "Master", "Mrs" etc.)

def replace_mean_age(name):
  # Gets everyone in the train dataset with a certain string in their name
  name_index_list = []
  for idx in range(len(df_train)):
    if name in str(df_train["Name"].iloc[[idx]]):
      name_index_list.append(idx)

  # Gets the ages of all those with the specified string in their name

  name_age_list = []
  for idx in range(len(name_index_list)):
    row = df_train.iloc[name_index_list[idx]]["Age"]
    name_age_list.append(row)

  # Same exact process but applied to the test dataset

  name_index_list_2 = []
  for idx in range(len(df_test)):
    if name in str(df_test["Name"].iloc[[idx]]):
      name_index_list_2.append(idx)

  name_age_list_2 = []
  for idx in range(len(name_index_list_2)):
    row = df_test.iloc[name_index_list_2[idx]]["Age"]
    name_age_list_2.append(row)
  
  # Joins both lists and removes any instances of nan

  name_age_list.extend(name_age_list_2)
  without_nan_age = [x for x in name_age_list if str(x) != "nan"]

  # Gets the mean age of everyone with the specified string in their name

  mean_of_name = mean_age(without_nan_age)

  '''
  If they have the string in their name, and if they have a "nan" age value,
  then their age value becomes the mean age of all the people who share their 
  specified string
  '''

  for idx in range(len(name_index_list)):
    if str(df_train.loc[name_index_list[idx], "Age"]) == "nan":
      df_train.loc[name_index_list[idx], "Age"] = mean_of_name

  for idx in range(len(name_index_list_2)):
    if str(df_test.loc[name_index_list_2[idx], "Age"]) == "nan":
      df_test.loc[name_index_list_2[idx], "Age"] = mean_of_name

  return mean_of_name

In [6]:
replace_mean_age("Master.")
replace_mean_age("Dr.")
replace_mean_age("Mrs.")
replace_mean_age("Miss.")
replace_mean_age("Mr.")


32.0

In [7]:
# Check for any remaining "nan" age values in train and test data

for idx in range(len(df_train)):
  if "NaN" in str(df_train["Age"].iloc[[idx]]):
    print(str(df_train["Age"].iloc[[idx]]))
    print(idx)

for idx in range(len(df_test)):
  if "NaN" in str(df_test["Age"].iloc[[idx]]):
    print(str(df_test["Age"].iloc[[idx]]))
    print(idx)

88   NaN
Name: Age, dtype: float64
88


In [8]:
# If there are still nan values, replace them with the mean of all ages (30.0).

df_train["Age"] = df_train["Age"].fillna(30.0)
df_test["Age"] = df_test["Age"].fillna(30.0)

In [43]:
# Replace male with 0 and female with 1 for skilearn 

df_train["Sex"] = df_train["Sex"].replace("male", 0)
df_train["Sex"] = df_train["Sex"].replace("female", 1)
df_test["Sex"] = df_test["Sex"].replace("male", 0)
df_test["Sex"] = df_test["Sex"].replace("female", 1)

In [14]:
train_data_x = df_train[["Pclass", "Sex", "Age"]]
train_data_x.head(10)

,Pclass,Sex,Age
0,3,0,22.0
1,1,1,38.0
2,3,1,26.0
3,1,1,35.0
4,3,0,35.0
5,3,0,32.0
6,1,0,54.0
7,3,0,2.0
8,3,1,27.0
9,2,1,14.0


In [15]:
train_data_y = df_train[["Survived"]]
train_data_y.head(10)

,Survived
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,0
8,1
9,1


In [16]:
test_data_x = df_test[["Pclass", "Sex", "Age"]]
test_data_x.head(30)

,Pclass,Sex,Age
0,3,0,34.5
1,3,1,47.0
2,2,0,62.0
3,3,0,27.0
4,3,1,22.0
5,3,0,14.0
6,3,1,30.0
7,2,0,26.0
8,3,1,18.0
9,3,0,21.0


In [28]:
clf1 = LogisticRegression()

In [29]:
clf2 = RandomForestClassifier(n_estimators=1000, max_features="sqrt", max_depth=None, min_samples_split=2)

In [30]:
clf3 = GaussianNB()

In [31]:
clf4 = KNeighborsClassifier(n_neighbors=9)

In [32]:
clf5 = SVC(kernel='rbf', probability=True)

In [33]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('knn', clf4), ('svc', clf5)], voting='hard')


In [34]:
clf1.fit(train_data_x, np.ravel(train_data_y))
clf2.fit(train_data_x, np.ravel(train_data_y))
clf3.fit(train_data_x, np.ravel(train_data_y))
clf4.fit(train_data_x, np.ravel(train_data_y))
clf5.fit(train_data_x, np.ravel(train_data_y))
eclf.fit(train_data_x, np.ravel(train_data_y))

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf',
                              RandomForestClassifier(max_features='sqrt',
                                                     n_estimators=1000)),
                             ('gnb', GaussianNB()),
                             ('knn', KNeighborsClassifier(n_neighbors=9)),
                             ('svc', SVC(probability=True))])

In [36]:
y_predict = eclf.predict(test_data_x.iloc[[26]])
print(y_predict)

[1]


In [37]:
predictions = []

In [38]:
for idx in range(len(test_data_x)):
  try:
    y_predict = eclf.predict(test_data_x.iloc[[idx]])
  except Exception:
    print(test_data_x.iloc[[idx]])
  y_predict = int(y_predict)
  predictions.append(y_predict)
print(predictions)
print(len(predictions))


[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

In [39]:
print(type(predictions[0]))

<class 'int'>


In [40]:
passengerids = df_test["PassengerId"].tolist()
print(passengerids)
print(len(passengerids))

[892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 107

In [41]:
dictionary_of_lists = {"PassengerId": passengerids, "Survived": predictions}
final_dataframe = pd.DataFrame(dictionary_of_lists)
final_dataframe.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [42]:
final_dataframe.to_csv("predictions5.csv", index=False)